In [1]:
import codecs
import json
import logging
import os
import sys
import operator
from collections import defaultdict
from tokenizer import twokenize
from twitter.Tweet import Tweet
from util import locations, ngrams
from processing import locationsInTweets
import stats_enriched_tweets

In [2]:
print os.getcwd()

/Users/cris/Documents/workspace/TwitterCrisis


In [3]:
# read enriched tweet
def tweetIter(inputfile):
    for line in codecs.open(inputfile, "r", "utf-8"):
        try:
            tweet = json.loads(line)
        except:
            print "Couldn't parse tweet: ", line[:200]
        yield tweet

In [4]:
def hasBothLocationAndMention(tweet):
    return (tweet["final_location"] or tweet["final_location_c"]) and (tweet["text_location_mentions"] or tweet["text_location_mentions_c"])

In [7]:
# Read file with simplified schema
'''
sentiment_tweet: STRING 'pro_ref' : pro refugees, 'against_ref' : against refugees 
                    (the tweet contains polarized hashtags)
sentiment_user: BOOLEAN  1 : pro refugees, 0 : against refugees 
                    (the user is polarized by analyzing the tweets he/she wrote so far)
'''


#inputFilename = "/Users/cris/all-brexit.json"
inputFilename = "/Users/cris/all-brexit-with-sent-lines.json"
tweetsAsDict = tweetIter(inputFilename)

# user sentiment count
countUserPro = 0 
countUserAgainst = 0

# tweet sentiment count
countTweetPro = 0
countTweetAgainst = 0

# user location - from final
userLocDict = defaultdict(int)
userLocCountryDict = defaultdict(int)
countFinalLoc = 0
countFinalLocCountry = 0

# location mention
locationMention = defaultdict(int)
locationCountryMention = defaultdict(int)
countLocMention = 0
countLocMention_c = 0

# both location and location mention
countBoth = 0
countBoth2 = 0


i = 0
for tweet in tweetsAsDict:
    i+=1
    
    if "sentiment_user" in tweet:
        if tweet["sentiment_user"] == 1:
            countUserPro += 1
        else:
            countUserAgainst += 1
            
            
    if "sentiment_tweet" in tweet:
        if tweet["sentiment_tweet"] == 'pro_ref':
            countTweetPro += 1
        else:
            countTweetAgainst += 1
            
    if tweet["final_location"]:
        countFinalLoc += 1
        userLocDict[tweet["final_location"]] +=1
            
    if tweet["final_location_c"]:
        countFinalLocCountry += 1
        userLocCountryDict[tweet["final_location_c"]] +=1
    
    if tweet["text_location_mentions"]:
        if len(tweet["text_location_mentions"]) > 1:
            for loc in tweet["text_location_mentions"]:
                countLocMention += 1
                locationMention[loc] +=1
        else:
            countLocMention += 1
            locationMention[tweet["text_location_mentions"][0]] +=1
            
    if tweet["text_location_mentions_c"]:
        if len(tweet["text_location_mentions_c"]) > 1:
            for loc_c in tweet["text_location_mentions_c"]:
                countLocMention_c += 1
                locationCountryMention[loc_c] +=1
        else:
            countLocMention_c += 1
            locationCountryMention[tweet["text_location_mentions_c"][0]] +=1
            
    if hasBothLocationAndMention(tweet):
        countBoth += 1
        
    if stats_enriched_tweets.hasUserLocationAndMentionLocation(tweet):
        countBoth2 +=1
    
#     if i%10000 ==0 :
#         break
        
print "Tweets with User pro: ", countUserPro, "Tweets with User against: ", countUserAgainst
print "Tweets pro: ", countTweetPro, "Tweets against: ", countTweetAgainst
print "Final loc: ", countFinalLoc, "Final loc country: ", countFinalLocCountry
print "Loc mention: ", countLocMention, "Loc mention country: ", countLocMention_c
print "With Loc and loc mention: ", countBoth, countBoth2

print i

Tweets with User pro:  632257 Tweets with User against:  195555
Tweets pro:  217492 Tweets against:  65934
Final loc:  856191 Final loc country:  1307711
Loc mention:  73325 Loc mention country:  221331
With Loc and loc mention:  199029 199029
1307711


In [6]:
print locationMention

defaultdict(<type 'int'>, {u'offenburg': 1, u'wallasey': 2160, u'craigavon': 1, u'brindisi': 13, u'tasikmalaya': 6, u'sambhal': 223, u'oss': 72, u'worms': 37, u'nottingham': 127, u'maastricht': 110, u'eastleigh': 7, u'tours': 50, u'mosul': 5, u'mila': 21, u'potenza': 1, u'grays': 1, u'kassel': 2, u'davao': 3, u'brugge': 1, u'venice': 26, u'shiraz': 2, u'marka': 3, u'lille': 6, u'paris': 3703, u'shanghai': 41, u'badajoz': 2, u'saint-denis': 1, u'canary wharf': 129, u'huddersfield': 31, u'littlehampton': 1, u'uga': 13, u'libertad': 70, u'chennai': 4, u'esher': 1, u'bracknell': 7, u'trani': 2, u'aschaffenburg': 1, u'lahore': 44, u'tolga': 86, u'welkom': 8, u'bratislava': 44, u'ankara': 453, u'gardez': 8, u'witten': 2, u'manado': 2, u'l\xfcneburg': 1, u'dar es salaam': 22, u'harlow': 26, u'morley': 10, u'osh': 6, u'murcia': 33, u'wakefield': 158, u'bournemouth': 32, u'los ba\xf1os': 1, u'bangor': 6, u'aachen': 75, u'sano': 12, u'kiel': 2, u'bergisch gladbach': 2, u'madurai': 1, u'horta-gui

In [7]:
a = sorted(locationCountryMention.items(), key=operator.itemgetter(1), reverse=True)

In [8]:
print "Countries: ", len(a), a

Countries:  151 [(u'united kingdom', 396155), (u'turkey', 34146), (u'ivory coast', 28814), (u'italy', 26016), (u'south africa', 23756), (u'germany', 19786), (u'france', 19083), (u'india', 18065), (u'nigeria', 13882), (u'ireland', 11644), (u'belgium', 9400), (u'spain', 9259), (u'china', 8774), (u'iraq', 8692), (u'israel', 7968), (u'gibraltar', 6301), (u'oman', 6206), (u'portugal', 5526), (u'indonesia', 4845), (u'japan', 4839), (u'croatia', 4262), (u'slovakia', 3908), (u'russia', 3390), (u'iran', 3116), (u'ghana', 3053), (u'norway', 2900), (u'bangladesh', 2896), (u'malaysia', 2439), (u'syria', 2135), (u'poland', 2113), (u'iceland', 1923), (u'greece', 1652), (u'pakistan', 1381), (u'hungary', 1319), (u'austria', 1294), (u'netherlands', 1292), (u'saudi arabia', 1177), (u'sweden', 1166), (u'south korea', 1031), (u'uganda', 932), (u'libya', 927), (u'switzerland', 871), (u'algeria', 846), (u'jordan', 815), (u'ukraine', 806), (u'kenya', 777), (u'denmark', 776), (u'philippines', 742), (u'singapo